In [17]:
import numpy as np
import math
import pandas as pd
from io import open
import unicodedata
import string
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import torch
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
!pip install wandb
import wandb
wandb.login(key = "b4f4cb721b8ca484302eddf5d87d6bb0be8b4f2c")
wandb.init(project="DL3_ASSN")
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

path ='/content/tel_train.csv'
df = pd.read_csv(path, header=None)
english_words = df[0]
Telugu_words = df[1]

path_val = '/content/tel_valid.csv'
df_val = pd.read_csv(path_val, header=None)
english_words_val = df_val[0]
Telugu_words_val = df_val[1]
# Extract English and Telugu words from data frames

path_test = '/content/tel_test.csv'
df_test = pd.read_csv(path_test, header=None)
english_words_test = df_test[0]
Telugu_words_test = df_test[1]

# Print rows from index 50000 onwards
print(df.iloc[50000:])

# Construct list of unique English characters and find maximum length of an English word
english_char_list = []
maxl_eng = -1
eng_word_length=0
for word in english_words:
    maxl_eng = max(maxl_eng, len(word))
    for char in word:
        english_char_list.append(char)
english_char_list = list(set(english_char_list))
print(f"English characters: {english_char_list}")
english_char_list.sort()
print(f"Maximum length of an English word: {maxl_eng}")

# Construct list of unique Telugu characters and find maximum length of a Telugu word
Telugu_char_list = []
maxl_tel = -1
word_length=0
for word in Telugu_words:
    maxl_tel = max(maxl_tel, len(word))
    for char in word:
        Telugu_char_list.append(char)
Telugu_char_list = list(set(Telugu_char_list))
print(f"Telugu characters: {Telugu_char_list}")
Telugu_char_list.sort()
print(f"Maximum length of a Telugu word: {maxl_tel}")

# Find maximum length of English and Telugu words in validation and test sets
for word in Telugu_words_test:
    maxl_tel = max(maxl_tel, len(word))
for word in Telugu_words_val:
    maxl_tel = max(maxl_tel, len(word))
print(f"Maximum length of a Telugu word in validation and test sets: {maxl_tel}")
for word in english_words_val:
    maxl_eng = max(maxl_eng, len(word))
for word in english_words_test:
    maxl_eng = max(maxl_eng, len(word))
print(f"Maximum length of an English word in validation and test sets: {maxl_eng}")
predictions_vanilla_test = []

# This function converts a word to a vector representation
def word2vec(word, lang):
    vec = []
    count=0
    if lang == "english":
        # Add 1 to represent unknown characters
        vec.append(len(english_char_list) + 1)
        for char in word:
            # Add 1 to represent character index (starting from 1)
            if char in english_char_list:
                vec.append(english_char_list.index(char) + 1)
                count+=1
            else:
                vec.append(len(english_char_list) + 1)# Represent unknown characters
                count+=1
        # Padding with 0s to match max length
        max_length=maxl_eng + 1
        while len(vec) < max_length:
            vec.append(0)
            count+=1
        # Add 0 at the end to represent end of word
        vec.append(0)
    else:
        # Add 1 to represent unknown characters
        vec.append(len(Telugu_char_list) + 1)
        for char in word:
            # Add 1 to represent character index (starting from 1)
            if char in Telugu_char_list:
                vec.append(Telugu_char_list.index(char) + 1)
            else:
                vec.append(len(Telugu_char_list) + 1) # Represent unknown characters
                count+=1
        # Padding with 0s to match max length
        while len(vec) < maxl_tel + 1:
            vec.append(0)
            count+=1
        # Add 0 at the end to represent end of word
        vec.append(0)
    return vec

vec = word2vec(Telugu_words[10],"Telugu")

def ip_matrix_construct(words, lang):
    temp=[]
    ans = []
    for word in words:
        count=0
        embedding = word2vec(word, lang)
        ans.append(embedding)
    return ans

# Construct input matrices for English and Telugu words
english_matrix = ip_matrix_construct(english_words, "english")
Telugu_matrix = ip_matrix_construct(Telugu_words, "Telugu")

# Print the number of English and Telugu words
print(f"Number of English words: {len(english_matrix)}")
print(f"Number of Telugu words: {len(Telugu_matrix)}")

# Convert input matrices to PyTorch tensors
english_matrix = torch.tensor(english_matrix)
Telugu_matrix = torch.tensor(Telugu_matrix)

# Print the dimensions of the English and Telugu input matrices
print(f"Dimensions of English matrix: {len(english_matrix)}, {len(english_matrix[0])}")
print(f"Dimensions of Telugu matrix: {len(Telugu_matrix)}, {len(Telugu_matrix[0])}")

# Construct input matrices for validation and test sets
english_matrix_val = ip_matrix_construct(english_words_val, "english")
english_matrix_val = torch.tensor(english_matrix_val)

english_matrix_test = ip_matrix_construct(english_words_test, "english")
english_matrix_test = torch.tensor(english_matrix_test)

Telugu_matrix_val = ip_matrix_construct(Telugu_words_val, "Telugu")
Telugu_matrix_val = torch.tensor(Telugu_matrix_val)

Telugu_matrix_test =ip_matrix_construct(Telugu_words_test, "Telugu")
Telugu_matrix_test = torch.tensor(Telugu_matrix_test)

class Encoder(nn.Module):
  def __init__(self,input_size, embedding_size, hidden_size, num_layers, p, cell_type, bidirectional,minimum):
    encoded_output=[]
    super(Encoder,self).__init__()
    self.cell_type = cell_type
    self.bidirectional = bidirectional
    self.dropout = nn.Dropout(p)
    self.embedding = nn.Embedding(input_size, embedding_size)
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    
    self.gru = nn.GRU(embedding_size, hidden_size, num_layers, dropout = p, bidirectional = bidirectional)
    self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p, bidirectional = bidirectional)
    self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout = p, bidirectional = bidirectional)
    array_words=[]
    
  def forward(self, x):
    embedding = self.dropout(self.embedding(x))
    if(self.cell_type == "GRU"):
      output, hidden = self.gru(embedding)
      return output, hidden
    if(self.cell_type == "RNN"):
      output, hidden = self.rnn(embedding)
      return output, hidden
    if(self.cell_type == "LSTM"):
      outputs, (hidden,cell) = self.lstm(embedding)
      return outputs,hidden, cell
    

  def initHidden(self):
    temp=self.hidden_size
    return torch.zeros(1, 1, temp, device=device)

class Decoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, p, cell_type,minimum):
    decoded_output=[]
    super(Decoder, self).__init__()

    self.dropout = nn.Dropout(p)
    self.cell_type = cell_type
    self.embedding = nn.Embedding(input_size, embedding_size)
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    #self.minimum=minimum
    self.fc = nn.Linear(hidden_size, output_size)  # fully connected.
    self.gru = nn.GRU(embedding_size, hidden_size, num_layers, dropout = p)
    self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout = p)
    self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)
    
    #self.minimum=minimum
  
  def forward(self,x,output, hidden, cell = 0,enco_out=None):
    temp,count=0
    x = x.unsqueeze(0).int()
    count+=1
    embedding = self.dropout(self.embedding(x))
    if(self.cell_type == "GRU"):
        outputs, hidden = self.gru(embedding, hidden)
        predictions = self.fc(outputs)
        predictions = predictions.squeeze(0)
        return predictions, hidden
    elif(self.cell_type == "RNN"):
        outputs, hidden = self.rnn(embedding, hidden)
        predictions = self.fc(outputs)
        predictions = predictions.squeeze(0)
        return predictions, hidden
    elif(self.cell_type == "LSTM"):
        outputs, (hidden, cell) = self.lstm(embedding, (hidden, cell))
        count+=1
        predictions = self.fc(outputs)
        predictions = predictions.squeeze(0)
        return predictions, hidden, cell

  def initHidden(self):
    temp=self.hidden_size
    return torch.zeros(1, 1, temp, device=device)

class Atten_decoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, p, cell_type, bidirectional,minimum):
    decoded_output=[]
    super(Atten_decoder, self).__init__()
    self.output_size = output_size
    self.dropout = nn.Dropout(p)
    self.cell_type = cell_type
    self.embedding = nn.Embedding(input_size, embedding_size)
    self.max_length = len(english_matrix[0])  
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    #self.minimum=minimum

    if(cell_type == "GRU"):
      self.gru = nn.GRU(hidden_size, hidden_size, num_layers, dropout = p)
    elif(cell_type == "RNN"):
      self.rnn = nn.RNN(hidden_size, hidden_size, num_layers, dropout = p)
    elif(cell_type == "LSTM"):
      self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, dropout = p)
    self.attn = nn.Linear(hidden_size+embedding_size, self.max_length)
    if bidirectional==False:
        self.attn_combine = nn.Linear(hidden_size +embedding_size, hidden_size)
    else:
        self.attn_combine = nn.Linear(hidden_size*2 +embedding_size, hidden_size)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x,output, hidden, cell = 0,enco_out=None):
    x = x.unsqueeze(0)
    count=0
    output=output.permute(1,0,2)
    # embedded = self.embedding(x)
    embedded = self.dropout(self.embedding(x))
    attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
    attn_applied = torch.bmm(attn_weights.unsqueeze(1),output).squeeze(1)
#     print("att applied done")
    output = torch.cat((embedded[0], attn_applied), 1)
#     print("concat done")
    output = F.relu(self.attn_combine(output).unsqueeze(0))
    if(self.cell_type == "GRU"):
        # print("GRU")
        outputs, hidden = self.gru(output, hidden)
        predictions = self.fc(outputs)
        predictions = predictions.squeeze(0)
        return predictions, hidden
    elif(self.cell_type == "RNN"):
        # print("GRU")
        outputs, hidden = self.rnn(output, hidden)
        predictions = self.fc(outputs)
        predictions = predictions.squeeze(0)
        return predictions, hidden
    elif(self.cell_type == "LSTM"):
        # print("GRU")
        outputs, (hidden, cell) = self.lstm(output, (hidden, cell))
        count+=1
        predictions = self.fc(outputs)
        predictions = predictions.squeeze(0)
        return predictions, hidden, cell

inp = []

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, cell_type, bidirectional, num_layers1,num_layers2,minimum):
        super(Seq2Seq, self).__init__()
        count=0
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
        self.bidirectional = bidirectional
        self.num_layers1 = num_layers1
        self.num_layers2 = num_layers2
        
    def forward(self, source, target, teacher_force_ratio=0.5):
        array_words2= []
        batch_size = source.shape[1]
        target_len = target.shape[0]
        temp_len=len(Telugu_char_list)
        target_vocab_size = temp_len + 2  
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        count=0
        if(self.cell_type == "LSTM"):
            encoder_output, hidden, cell = self.encoder(source)
            count+=1
        else:
            encoder_output, hidden = self.encoder(source)
            count+=1
        if(self.num_layers1 != self.num_layers2 or self.bidirectional == True):
          temp1=0
          temp = hidden[self.num_layers1 - 1] + hidden[self.num_layers1 - 1]
          hidden=temp
          temp2 = hidden.repeat(self.num_layers2,1,1)
          hidden=temp2
          if(self.cell_type == "LSTM"):
              temp_2=0
              c_temp = cell[self.num_layers1 - 1] + cell[self.num_layers1 - 1]
              cell=c_temp
              c_temp2 = cell.repeat(self.num_layers2,1,1)
              cell=c_temp2
        x = target[0]    
        for t in range(0, target_len-1):
            if(self.cell_type == "LSTM"):
                hidden_temp=[]
                output, hidden, cell = self.decoder(x, encoder_output,hidden, cell)
                hidden_temp=hidden
            else :
                output, hidden = self.decoder(x,encoder_output, hidden)
            outputs[t+1] = output
            best_guess = output.argmax(1)
            x = target[t+1] if random.random() < teacher_force_ratio else best_guess
        return outputs

def Accuracy(model, english_matrix, Telugu_matrix, epoch, batch_size):
    count = 0
    temp=len(english_matrix)
    for batch_idx in range(1,(int)(temp/ batch_size)+1):
        inp_data = english_matrix[batch_size * (batch_idx-1) : batch_size * ((batch_idx-1)+1)].to(device)
        target = Telugu_matrix[batch_size * (batch_idx-1) : batch_size * ((batch_idx-1)+1)].to(device)
        count+=1
        output = model.forward(inp_data.T, target.T, 0)
        output1=output.T
        output = nn.Softmax(dim=2)(output)
        output2= output.T
        output = torch.argmax(output, dim=2)
        output = output.T
        for i in range(1,batch_size+1):
            if(torch.equal(output[i-1][1:],target[i-1][1:])):
                count += 1
    return count * 100 / len(english_matrix)

# creating list of expected marathi word and predicted marathi word.
def word_prediction(model, english_matrix, telugu_matrix, batch_size):
  inv=(int)(len(english_matrix) / batch_size)
  for batch_idx in range(inv):
        count=0
        bs=batch_size
        inp_data = english_matrix[bs * batch_idx : bs * (batch_idx+1)].to(device)
        target = telugu_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        output = model.forward(inp_data.T, target.T, 0)
        prediction=[]
        output = nn.Softmax(dim=2)(output)
        output = torch.argmax(output, dim=2)
        output = output.T
        lp=len(inp_data)
        for i in range(1,lp+1):
          word = ""
          count+=1
          inp_word = inp_data[i-1]
          for j in range(len(inp_word)):
            eng_char_array_size=27
            if(inp_word[j]>0 and inp_word[j]<eng_char_array_size):
              temp_inp=english_char_list[inp_word[j]-1]
              word += temp_inp
          inp.append(word)
        for i in range(len(target)):
          count+=1
          word1 = ""
          word2 = ""
          target_word = target[i]
          output_word = output[i]
          prediction.append(output_word)
          for j in range(1,len(target_word)+1):
            if(target_word[j-1]>0 and target_word[j-1]<63):
              temp1 = Telugu_char_list[target_word[j-1] - 1]
              word1 += temp1
          count1=count
          for j in range(1,len(output_word)+1):
            if(output_word[j-1]>0 and output_word[j-1]<63):
              temp2=Telugu_char_list[output_word[j-1] - 1]
              word2 = word2 + temp2
          temp = [word1, word2]        
  df_test = pd.DataFrame({"Input": inp, "Target" : temp[0], "Predicted":temp[1]})
  df_test.to_csv("Deepak_vanilla.csv", index=False)

def neural_network(cell_type, bidirectional, num_layers1,num_layers2, batch_size, embedding_size, hidden_size, num1_dropout,num2_dropout,attention):
  num_epochs = 1
  learning_rate = 0.001
  minimum=0
  count=0
  eng_temp=len(english_char_list) + 2 
  mar_temp=len(Telugu_char_list) + 2
  input_size_encoder = eng_temp # intially= 30 || Now 26 + 1(start token) + 1 = 28 
  input_size_decoder = mar_temp  # intitially = 66 || Now 63 + 1(start token) + 1 = 65
  output_size        = mar_temp  # intitially = 66 || Now 63 + 1(start token) + 1 = 65

  encoder_net = Encoder(input_size_encoder, embedding_size, hidden_size, num_layers1, num1_dropout, cell_type,bidirectional,minimum).to(device)
  if(attention):
    decoder_net = Atten_decoder(input_size_decoder,embedding_size,hidden_size,output_size,num_layers2,num2_dropout, cell_type, bidirectional,minimum).to(device)
  else:
    decoder_net = Decoder(input_size_decoder,embedding_size,hidden_size,output_size,num_layers2,num2_dropout, cell_type,minimum).to(device)

  model = Seq2Seq(encoder_net, decoder_net, cell_type, bidirectional, num_layers1, num_layers2,minimum).to(device)
  temp_l= len(Telugu_char_list) 
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  pad_idx = temp_l+1 # pading index for Telugu
  criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

  for epoch in range(1,num_epochs+1):
      print("epoch = ",epoch)
      model.train()
      total_loss = 0
      step = 0
      val_loss=0
      for batch_idx in range(1,((int)(len(english_matrix) / batch_size))+1):
          inp_data = english_matrix[batch_size * (batch_idx-1) : batch_size * ((batch_idx-1)+1)].to(device)
          target = Telugu_matrix[batch_size * (batch_idx-1) : batch_size * ((batch_idx-1)+1)].to(device)
          target = target.T
          val_loss=0
          output = model(inp_data.T, target)
          output = output[1:].reshape(-1, output.shape[2])
          output_fnl=output[1:]
          target = target[1:].reshape(-1)
          optimizer.zero_grad()
          loss = criterion(output, target)
          val_loss+= loss/step
          total_loss += loss
          # Back prop
          loss.backward()
          val_loss=val_loss/100
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

          # Gradient descent step
          optimizer.step()

          step += 1
          count += 1
      
      training_accuracy = Accuracy(model, english_matrix, Telugu_matrix, epoch, batch_size)
      val_accuracy = Accuracy(model, english_matrix_val, Telugu_matrix_val, epoch, batch_size)
      print("total loss = ",total_loss / step)
      print("Training Accuracy = ", training_accuracy)
      print("Validation accuracy = ",val_accuracy)
  word_prediction(model, english_matrix_test, Telugu_matrix_test, batch_size,)
  wandb.log({"train_accuracy": training_accuracy, "validation_accuracy": val_accuracy, "training_loss": total_loss / step,  'epoch': epoch})

# neural_network("LSTM", True, 1, 1, 64, 256, 512, 0.3, 0.3, True)

df_test = pd.read_csv("Deepak_vanilla.csv")
df_test = df_test.sample(frac=1)
df_test.head(20)
sweep_config = {
  "name": "Assignment_3",
  "method": "bayes",
  "metric": {
      "name":"validation_accuracy",
      "goal": "maximize"
  },
  "parameters": {
        "num_layers": {
            "values": [1, 2, 3]
        },
        "num_layers2": {
            "values": [1, 2, 3]
        },
        "bidirectional": {
            "values": [True, False]
        },

        "attention": {
            "values": [True, False]
        },
        "hidden_size": {
            "values": [16,32,64,128, 256, 512]
        },
        "batch_size": {
            "values": [32,64,128]
        },
        "dropout": {
            "values": [0.2,0.3,0.4,0.5]
        },
        "num2_dropout": {
            "values": [0.2,0.3,0.4,0.5]
        },
        "embedding_size": {
            "values": [16, 32, 64,128, 256, 512]
        },
        
        "cell_type": {
            "values": ["GRU", "RNN", "LSTM"]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, entity="deepak3103", project="DL3_ASSN")

def Train():
    # Initialize a new wandb run
    wandb.init()
    wandb.run.name = "ct_" + str(wandb.config.cell_type) +"_bd_" + str(wandb.config.bidirectional)  + "_NL_"+str(wandb.config.num_layers)+ "_BS_" + str(wandb.config.batch_size)+ "_HS_" + str(wandb.config.hidden_size) +"_ES_" + str(wandb.config.embedding_size)+ "_DO_" + str(wandb.config.num1_dropout)
    print(wandb.run.name)
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    embedding_size = config.embedding_size
    hidden_size = config.hidden_size
    intial_epoch=0
    cell_type = config.cell_type
    num_layers1 = config.num_layers
    num_layers2 = config.num_layers
    batch_size = config.batch_size
    num1_dropout = config.dropout
    num2_dropout=config.dropout
    bidirectional = config.bidirectional
    attention=config.attention
    intial_run=0

    neural_network(cell_type, bidirectional, num_layers1, num_layers2, batch_size, embedding_size, hidden_size, num1_dropout, num2_dropout, attention)
    wandb.run.save()

wandb.agent(sweep_id, Train , count=100)

                            0                   1
50000               repinundi           రేపినుండి
50001               aavutundi            ఆవుతుంది
50002            vrukshamtone          వృక్షంతోనే
50003          mosukupogaladu        మోసుకుపోగలదు
50004                seligman           సెలిగ్మన్
...                       ...                 ...
51195            telustondavi        తెలుస్తోందవి
51196                patching             పేచింగ్
51197  venakkiteesukoovaalane  వెనక్కితీసుకోవాలనే
51198          roopaantaraalu          రూపాంతరాలు
51199            chendindindi         చెందిందింది

[1200 rows x 2 columns]
English characters: ['o', 'c', 'r', 'u', 'y', 'h', 'x', 'e', 's', 'q', 'j', 'p', 'v', 'z', 'w', 'f', 'g', 'b', 't', 'm', 'l', 'd', 'n', 'a', 'k', 'i']
Maximum length of an English word: 28
Telugu characters: ['స', 'ు', 'ం', 'ై', 'ృ', 'జ', 'త', 'ఋ', 'ళ', 'ఠ', 'ఱ', 'ణ', 'డ', 'ప', 'ఇ', 'హ', 'అ', 'ీ', 'ఢ', 'ి', 'ూ', 'భ', 'వ', 'ఔ', 'ఛ', 'ఝ', 'ే', 'య', 'ఊ', 'ద', 'గ', 'మ', 'చ', '

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


epoch =  1
total loss =  tensor(0.4916, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  50.328125
Validation accuracy =  46.09375


,Input,Target,Predicted
3169,bhatpara,పేడ,పెడా
920,vyaas,పేడ,పెడా
2360,thamaku,పేడ,పెడా
1738,viruddangaa,పేడ,పెడా
1797,glaasullone,పేడ,పెడా
1242,erickson,పేడ,పెడా
3000,saagutondani,పేడ,పెడా
787,vuntaayi,పేడ,పెడా
2618,vaasudevaraavu,పేడ,పెడా
2908,barai,పేడ,పెడా
